In [10]:
!git clone https://github.com/aws-samples/llmops-workshop/

Cloning into 'llmops-workshop'...
remote: Enumerating objects: 221, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (181/181), done.
remote: Total 221 (delta 70), reused 175 (delta 33), pack-reused 0
Receiving objects: 100% (221/221), 4.61 MiB | 42.90 MiB/s, done.
Resolving deltas: 100% (70/70), done.


In [ ]:
%pip install sagemaker boto3 datasets pygments -U -q

In [54]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [47]:

import sagemaker
from sagemaker.local import LocalSession
from sagemaker.experiments.run import Run
from sagemaker.huggingface import HuggingFace, HuggingFaceProcessor, HuggingFaceModel, get_huggingface_llm_image_uri
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role
from sagemaker.pytorch.processing import PyTorchProcessor

import json

import boto3
import os
from datetime import datetime
import uuid
from datasets import load_dataset

import time


# config

In [4]:
sess = sagemaker.Session()
region = sess.boto_region_name
sm_client = boto3.client("sagemaker")

# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']


print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {region}")


sagemaker role arn: arn:aws:iam::133114324038:role/service-role/AmazonSageMaker-ExecutionRole-20240406T005817
sagemaker bucket: sagemaker-us-east-1-133114324038
sagemaker session region: us-east-1


In [5]:
if "base_model_pkg_group_name" not in locals():
    base_model_pkg_group_name = "None"

In [6]:
rand_id = uuid.uuid4().hex[:5] # this is the random-id assigned for each run. 
training_dataset_s3_loc = f"s3://{sagemaker_session_bucket}/data/workshop-{rand_id}/train"
validation_dataset_s3_loc = f"s3://{sagemaker_session_bucket}/data/workshop-{rand_id}/eval"
model_output_s3_loc = f"s3://{sagemaker_session_bucket}/data/workshop-{rand_id}/model"
model_eval_s3_loc = f"s3://{sagemaker_session_bucket}/data/workshop-{rand_id}/modeleval"
model_id = "NousResearch/Llama-2-7b-chat-hf"
hf_dataset_name = "hotpot_qa"

print(f"training_dataset_s3_loc: {training_dataset_s3_loc}")
print(f"validation_dataset_s3_loc: {validation_dataset_s3_loc}")
print(f"model artifact S3 location: {model_output_s3_loc}")
print(f"model evaluation output S3 location: {model_eval_s3_loc}")
print(f"model_id: {model_id}")
print(f"base model package group name: {base_model_pkg_group_name}")
print(f"Huggingfae dataset name: {hf_dataset_name}")

training_dataset_s3_loc: s3://sagemaker-us-east-1-133114324038/data/workshop-5449f/train
validation_dataset_s3_loc: s3://sagemaker-us-east-1-133114324038/data/workshop-5449f/eval
model artifact S3 location: s3://sagemaker-us-east-1-133114324038/data/workshop-5449f/model
model evaluation output S3 location: s3://sagemaker-us-east-1-133114324038/data/workshop-5449f/modeleval
model_id: NousResearch/Llama-2-7b-chat-hf
base model package group name: None
Huggingfae dataset name: hotpot_qa


# Data 

In [16]:
# template dataset to add prompt to each sample
def template_dataset(sample):
    """
    Create a field for the given sample to store formatted llama2 prompt.
    
    @type  sample: Dataset sample
    @param sample: Dataset sample
    @rtype:   Dataset
    @return:  Dataset sample
    
    """
    sample["text"] = f"{format_hotpot(sample)}"
    return sample

In [21]:
def format_hotpot(sample):
    """
    Function that takes a single data sample derived from Huggingface datasets API: (https://huggingface.co/docs/datasets/index)
    and formats it into llama2 prompt format. For more information about llama2 prompt format, 
    please refer to https://huggingface.co/blog/llama2#how-to-prompt-llama-2 
    
    An example prompt is shown in the following:
    <s>
      [INST] <<SYS>>
        {{system}}
      <</SYS>>

      ### Question
      {{question}}

      ### Context
      {{context}}[/INST] {{answer}}</s>
    
    @type  sample: Dataset sample
    @param sample: dataset sample
    @rtype:   string
    @return:  llama2 prompt format
    """
    
    prefix = "<s>"
    postfix = "</s>"
    system_start_tag = "<<SYS>>"
    system_end_tag = "<</SYS>>"
    instruction_start_tag = "[INST]"
    instruction_end_tag = "[/INST]"
    context = "\n".join([ "".join(x) for x in sample['context']['sentences']])
    system = f"Given the following context, answer the question as accurately as possible:"
    question_prompt = f"### Question\n{sample['question']}"
    context_prompt = f"### Context\n{context}"
    prompt = f"{prefix}\n{instruction_start_tag} {system_start_tag}\n{system}\n{system_end_tag}\n\n{question_prompt}\n\n{context_prompt}{instruction_end_tag} {sample['answer']} {postfix}" 
    return prompt

In [28]:
data_path = '/home/sagemaker-user/data/'
dataset = load_dataset(hf_dataset_name, "distractor", split=f"train[:10%]")
new_dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))
training_input_path = data_path + "/train"
new_dataset.save_to_disk(training_input_path)

Saving the dataset (0/1 shards):   0%|          | 0/9045 [00:00<?, ? examples/s]

In [30]:
eval_dataset = load_dataset(hf_dataset_name, "distractor", split=f"train[:10%]")
new_eval_dataset = dataset.map(template_dataset, remove_columns=list(eval_dataset.features))
eval_input_path = data_path + "/eval"
new_eval_dataset.save_to_disk(eval_input_path)


Saving the dataset (0/1 shards):   0%|          | 0/9045 [00:00<?, ? examples/s]

In [38]:
torch_processor = PyTorchProcessor(
    framework_version='2.0',
    role=get_execution_role(),
    instance_type='ml.m5.xlarge',
    # instance_type='local', # uncomment for local mode
    instance_count=1,
    base_job_name='frameworkprocessor-PT',
    py_version="py310",
    sagemaker_session=sess
)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


In [45]:

torch_processor.run(
    code="preprocess.py",
    source_dir="/home/sagemaker-user/llmops-workshop/src/preprocess",
    outputs=[
        ProcessingOutput(output_name="train_data",
                         source="/opt/ml/processing/train",
                         destination=training_dataset_s3_loc),
        ProcessingOutput(output_name="eval_data",
                         source="/opt/ml/processing/eval",
                         destination=validation_dataset_s3_loc),

    ],
    arguments=["--train-data-split", "1:50",
               "--eval-data-split", "51:100",
               "--hf-dataset-name", hf_dataset_name]
)

INFO:sagemaker.processing:Uploaded /home/sagemaker-user/llmops-workshop/src/preprocess to s3://sagemaker-us-east-1-133114324038/frameworkprocessor-PT-2024-04-21-13-26-23-667/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-133114324038/frameworkprocessor-PT-2024-04-21-13-26-23-667/source/runproc.sh
INFO:sagemaker:Creating processing-job with name frameworkprocessor-PT-2024-04-21-13-26-23-667


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.4
    Uninstalling huggingface-hub-0.19.4:
      Successfully uninstalled huggingface-hub-0.19.4
[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Received arguments Namespace(hf_dataset_name='hotpot_qa', train_data_split='1:50', eval_data_split='51:100')
#015Downloading data:   0%|          | 0.00/301M [00:00<?, ?B/s]#015Downloading data:  10%|█         | 31.5M/301M [00:00<00:01, 266MB/s]#015Downloading data:  28%|██▊       | 83.9M/301M [00:00<00:00, 365MB/s]#015Downloading data:  45%|████▌     | 136M/301M [00:00<00:00, 398MB/s] #015Downloading data:  59%|█████▉    | 178M/301M [00:00<00:00, 382M

# Finetune

In [46]:
!pygmentize /home/sagemaker-user/llmops-workshop/src/train/train.py

import os
import torch
import argparse
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from datasets import load_from_disk
import tarfile
import boto3
import sagemaker
from sagemaker.huggingface import HuggingFaceModel
from sagemaker.collection import Collection
from tqdm import tqdm
from smexperiments_callback import SageMakerExperimentsCallback
import pathlib
from botocore.exceptions import ClientError
import logging
from urllib.parse import urlparse
import json
import shutil
import bitsandbytes as bnb


# Output directory where the model predictions and checkpoints will be stored
output_dir = "/opt/ml/model/"
base_model_path = "/tmp/basemodel"
model_eval_save_dir = "/tmp/eval"

# Load the entire model on the GPU 0
device_map = "auto"

def parse_arge():
    """Parse the arguments."""
    parser = ar

In [48]:

# define Training Job Name 
time_suffix = datetime.now().strftime('%y%m%d%H%M')
experiments_name = f"exp-{model_id.replace('/', '-')}"
run_name = f"qlora-finetune-run-{time_suffix}-{rand_id}"

# define Training Job Name 
job_name = f'huggingface-qlora-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}-{rand_id}'


In [50]:
# hyperparameters, which are passed into the training job
hyperparameters ={
  'model_id': model_id,                                # pre-trained model
  'epochs': 1,                                         # number of training epochs
  'per_device_train_batch_size': 8,                    # Batch size per GPU for training
  'per_device_eval_batch_size' : 8,                    # Batch size per GPU for evaluation
  'learning_rate' : 2e-4,                              # Initial learning rate (AdamW optimizer)
  'optimizer' : "paged_adamw_32bit",                   # Optimizer to use
  'logging_steps' : 5,                                 # Log every X updates steps
  'lora_r': 64,                                        # LoRA attention dimension.
  'lora_alpha' : 16,                                   # The alpha parameter for Lora scaling
  'lora_dropout' : 0.1,                                # The dropout probability for Lora layers
  'use_4bit' : True,                                   # Activate 4-bit precision base model loading
  'bnb_4bit_compute_dtype' : "float16",                # Compute dtype for 4-bit base models
  'bnb_4bit_quant_type' : "nf4",                       # Quantization type (fp4 or nf4)
  'base_model_group_name' : base_model_pkg_group_name, # Base model registered in SageMaker Model Registry
  'region': region,                                    # AWS region where the training is run
  'model_eval_s3_loc' : model_eval_s3_loc              # S3 location for uploading the model evaluation metrics
}

print(f"SageMaker experiment name: {experiments_name}")
print(f"SageMaker experiment run name: {run_name}")
print(f"SageMaker training job name: {job_name}")

SageMaker experiment name: exp-NousResearch-Llama-2-7b-chat-hf
SageMaker experiment run name: qlora-finetune-run-2404211333-5449f
SageMaker training job name: huggingface-qlora-2024-04-21-13-33-09-5449f


# Run the training job

In [56]:
with Run(
    experiment_name=experiments_name,
    run_name=run_name,
    sagemaker_session=sess
) as run:

    # create the Estimator
    huggingface_estimator = HuggingFace(
        entry_point='train.py',         # train script
        source_dir='/home/sagemaker-user/llmops-workshop/src/train',         # directory which includes all the files needed for training
        instance_type='ml.g5.xlarge', # instances type used for the training job
        # instance_type='local_gpu',      # use local 
        instance_count=1,               # the number of instances used for training
        base_job_name=job_name,         # the name of the training job
        role=get_execution_role(),      # Iam role used in training job to access AWS ressources, e.g. S3
        volume_size= 200,    # the size of the EBS volume in GB
        transformers_version='4.28.1',    # the transformers version used in the training job
        pytorch_version='2.0.0',          # the pytorch_version version used in the training job
        py_version='py310',             # the python version used in the training job
        hyperparameters= hyperparameters,
        environment={ "HUGGINGFACE_HUB_CACHE": "/tmp/.cache" }, # set env variable to cache models in /tmp
        sagemaker_session=sess,         # specifies a sagemaker session object
        output_path=model_output_s3_loc # s3 location for model artifact,
    )
    
    # define a data input dictonary with our uploaded s3 uris
    data = { 'training': training_dataset_s3_loc,
             'validation': validation_dataset_s3_loc}

    # starting the train job with our uploaded datasets as input
    huggingface_estimator.fit(data, wait=True)
    run.log_parameters(data)

INFO:sagemaker.experiments.run:The run (qlora-finetune-run-2404211333-5449f) under experiment (exp-nousresearch-llama-2-7b-chat-hf) already exists. Loading it.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-qlora-2024-04-21-13-33-09-5-2024-04-21-13-43-39-299


2024-04-21 13:43:39 Starting - Starting the training job...
2024-04-21 13:43:54 Starting - Preparing the instances for training...
2024-04-21 13:44:31 Downloading - Downloading input data...
2024-04-21 13:44:56 Downloading - Downloading the training image..............................
2024-04-21 13:49:47 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-04-21 13:49:57,219 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-04-21 13:49:57,238 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-04-21 13:49:57,248 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-04-21 13:49:57,255 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-04-21 13:49:58,976 sagemaker-training-toolkit INFO     Installing de

# Deploy

In [59]:

llm_image = sagemaker.image_uris.retrieve(
    "djl-deepspeed", region=region, version="0.23.0"
)

# print ecr image uri
print(f"llm image uri: {llm_image}")

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


llm image uri: 763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118


In [60]:
# sagemaker config
instance_type = "ml.g5.2xlarge"
number_of_gpu = 1
health_check_timeout = 3600


In [61]:
# create HuggingFaceModel with the a DJI image uri
huggingface_model = HuggingFaceModel(
    model_data=huggingface_estimator.model_data,
    image_uri=llm_image,
    transformers_version="4.28.1",
    pytorch_version="2.0.0",
    py_version="py310",
    model_server_workers=1,
    role=role,
    sagemaker_session=sess,
)

In [62]:
endpoint_name_random_id = uuid.uuid4().hex[:5]
endpoint_name = f"llama2-7b-djl-deepspeed-{endpoint_name_random_id}"

print(f"endpoint name: {endpoint_name}")
llm = huggingface_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=health_check_timeout, 
  endpoint_name=endpoint_name
)

INFO:sagemaker:Creating model with name: djl-inference-2024-04-21-14-05-22-248


endpoint name: llama2-7b-djl-deepspeed-ec66e


INFO:sagemaker:Creating endpoint-config with name llama2-7b-djl-deepspeed-ec66e
INFO:sagemaker:Creating endpoint with name llama2-7b-djl-deepspeed-ec66e


--------------!

# Test the endpoint

In [65]:
prompt_template = """<s>
[INST] <<SYS>>
{{system}}
<</SYS>>

### Question
{{question}}

### Context
{{context}}[/INST] """

In [66]:
system_message = "Given the following context, answer the question as accurately as possible:"

def build_llama2_prompt(message):
    question = message['question']
    context = message['context']
    formatted_message = prompt_template.replace("{{system}}", system_message)
    formatted_message = formatted_message.replace("{{question}}", question)
    formatted_message = formatted_message.replace("{{context}}", context)
    return formatted_message

In [67]:
message = {}
message['question'] = "The Oberoi family is part of a hotel company that has a head office in what city?"
message['context'] = """The Ritz-Carlton Jakarta is a hotel and skyscraper in Jakarta, Indonesia and 14th Tallest building in Jakarta. It is located in city center of Jakarta, near Mega Kuningan, adjacent to the sister JW Marriott Hotel. It is operated by The Ritz-Carlton Hotel Company. The complex has two towers that comprises a hotel and the Airlangga Apartment respectively. The hotel was opened in 2005.
The Oberoi family is an Indian family that is famous for its involvement in hotels, namely through The Oberoi Group.
The Oberoi Group is a hotel company with its head office in Delhi. Founded in 1934, the company owns and/or operates 30+ luxury hotels and two river cruise ships in six countries, primarily under its Oberoi Hotels & Resorts and Trident Hotels brands.
The 289th Military Police Company was activated on 1 November 1994 and attached to Hotel Company, 3rd Infantry (The Old Guard), Fort Myer, Virginia. Hotel Company is the regiment\'s specialty company.\nThe Glennwanis Hotel is a historic hotel in Glennville, Georgia, Tattnall County, Georgia, built on the site of the Hughes Hotel. The hotel is located at 209-215 East Barnard Street. The old Hughes Hotel was built out of Georgia pine circa 1905 and burned in 1920. The Glennwanis was built in brick in 1926. The local Kiwanis club led the effort to get the replacement hotel built, and organized a Glennville Hotel Company with directors being local business leaders. The wife of a local doctor won a naming contest with the name "Glennwanis Hotel", a suggestion combining "Glennville" and "Kiwanis".'"""

In [72]:
message = {}
message['question'] = "Who ordered the wiretapping of Democratic Party headquarters"
message['context'] = """On January 27, 1972, G. Gordon Liddy, Finance Counsel for the Committee for the Re-Election of the President (CRP) and former aide to John Ehrlichman, presented a campaign intelligence plan to CRP's acting chairman Jeb Stuart Magruder, Attorney General John Mitchell, and Presidential Counsel John Dean that involved extensive illegal activities against the Democratic Party. According to Dean, this marked "the opening scene of the worst political scandal of the twentieth century and the beginning of the end of the Nixon presidency".[19]: p. xvii 
Mitchell viewed the plan as unrealistic. Two months later, Mitchell approved a reduced version of the plan, including burglarizing the Democratic National Committee's (DNC) headquarters at the Watergate Complex in Washington, D.C. to photograph campaign documents and install listening devices in telephones. Liddy was nominally in charge of the operation,[citation needed] but has since insisted that he was duped by both Dean and at least two of his subordinates, which included former CIA officers E. Howard Hunt and James McCord, the latter of whom was serving as then-CRP Security Coordinator after John Mitchell had by then resigned as attorney general to become the CRP chairman.[20][21]
In May, McCord assigned former FBI agent Alfred C. Baldwin III to carry out the wiretapping and monitor the telephone conversations afterward.[22] McCord testified that he selected Baldwin's name from a registry published by the FBI's Society of Former Special Agents to work for the Committee to Re-elect President Nixon. Baldwin first served as bodyguard to Martha Mitchell—John Mitchell's wife, who was living in Washington. Baldwin accompanied Martha Mitchell to Chicago. Eventually the committee replaced Baldwin with another security man.[citation needed]
On May 11, McCord arranged for Baldwin, whom investigative reporter Jim Hougan described as "somehow special and perhaps well known to McCord", to stay at the Howard Johnson's motel across the street from the Watergate complex.[23] Room 419 was booked in the name of McCord's company.[23] At the behest of Liddy and Hunt, McCord and his team of burglars prepared for their first Watergate break-in, which began on May 28.[24]
Two phones inside the DNC headquarters' offices were said to have been wiretapped.[25] One was Robert Spencer Oliver's phone. At the time, Oliver was working as the executive director of the Association of State Democratic Chairmen. The other phone belonged to DNC chairman Larry O'Brien. The FBI found no evidence that O'Brien's phone was bugged;[26] however, it was determined that an effective listening device was installed in Oliver's phone. While successful with installing the listening devices, the committee agents soon determined that they needed repairs. They plotted a second "burglary" in order to take care of the situation.[25]
Sometime after midnight on Saturday, June 17, 1972, Watergate Complex security guard Frank Wills noticed tape covering the latches on some of the complex's doors leading from the underground parking garage to several offices, which allowed the doors to close but stay unlocked. He removed the tape, believing it was nothing.[27] When he returned a short time later and discovered that someone had retaped the locks, he called the police.[27]
Police dispatched an unmarked police car with three plainclothes officers, Sgt. Paul W. Leeper, Officer John B. Barrett, and Officer Carl M. Shoffler, who were working the overnight shift; they were often referred to as the "bum squad" because they often dressed undercover as hippies and were on the lookout for drug deals and other street crimes.[28] Alfred Baldwin, on "spotter" duty at the Howard Johnson's hotel across the street, was distracted watching the film Attack of the Puppet People on TV and did not observe the arrival of the police car in front of the Watergate building,[28] nor did he see the plainclothes officers investigating the DNC's sixth floor suite of 29 offices. By the time Baldwin finally noticed unusual activity on the sixth floor and radioed the burglars, it was already too late.[28]
The police apprehended five men, later identified as Virgilio Gonzalez, Bernard Barker, James McCord, Eugenio Martínez, and Frank Sturgis.[20] They were criminally charged with attempted burglary and attempted interception of telephone and other communications. The Washington Post reported the day after the burglary that, "police found lock-picks and door jimmies, almost $2,300 in cash, most of it in $100 bills with the serial numbers in sequence ... a shortwave receiver that could pick up police calls, 40 rolls of unexposed film, two 35 millimeter cameras and three pen-sized tear gas guns".[29] The Post would later report that the actual amount of cash was $5,300.[30]
The following morning, Sunday, June 18, G. Gordon Liddy called Jeb Magruder in Los Angeles and informed him that "the four men arrested with McCord were Cuban freedom fighters, whom Howard Hunt recruited". Initially, Nixon's organization and the White House quickly went to work to cover up the crime and any evidence that might have damaged the president and his reelection.'On September 15, 1972, a grand jury indicted the five office burglars, as well as Hunt and Liddy,[32] for conspiracy, burglary, and violation of federal wiretapping laws. The burglars were tried by a jury, with Judge John Sirica officiating, and pled guilty or were convicted on January 30, 1973.[33]"""



In [73]:
input = build_llama2_prompt(message)

In [74]:
print(input)

<s>
[INST] <<SYS>>
Given the following context, answer the question as accurately as possible:
<</SYS>>

### Question
Who ordered the wiretapping of Democratic Party headquarters

### Context
On January 27, 1972, G. Gordon Liddy, Finance Counsel for the Committee for the Re-Election of the President (CRP) and former aide to John Ehrlichman, presented a campaign intelligence plan to CRP's acting chairman Jeb Stuart Magruder, Attorney General John Mitchell, and Presidential Counsel John Dean that involved extensive illegal activities against the Democratic Party. According to Dean, this marked "the opening scene of the worst political scandal of the twentieth century and the beginning of the end of the Nixon presidency".[19]: p. xvii 
Mitchell viewed the plan as unrealistic. Two months later, Mitchell approved a reduced version of the plan, including burglarizing the Democratic National Committee's (DNC) headquarters at the Watergate Complex in Washington, D.C. to photograph campaign doc

In [75]:
params = {
    "do_sample": True,
    "top_p": 0.9,
    "temperature": 0.9,
    "top_k": 50,
    "max_new_tokens": 512,
    "repetition_penalty": 1.03,
  }

output = llm.predict({"text":input, "properties" : params})

In [76]:
print(output['outputs'][0]["generated_text"][len(input):]) # automatically removed the bos_token and eos_token_id

 Based on the information provided in the text, it appears that the order to wiretap Democratic Party headquarters came from high-ranking officials within the Nixon administration, including Attorney General John Mitchell and Presidential Counsel John Dean. However, the exact extent of their involvement and the specific individuals who authorized the wiretapping are not clearly stated in the passage.

According to the text, G. Gordon Liddy, a lawyer and former aide to John Ehrlichman, presented a campaign intelligence plan to CRP's acting chairman Jeb Stuart Magruder, Attorney General John Mitchell, and Presidential Counsel John Dean in January 1972. This plan included extensive illegal activities against the Democratic Party, including wiretapping their headquarters. Two months later, Mitchell approved a reduced version of the plan, which included burglarizing the Democratic National Committee's (DNC) headquarters at the Watergate Complex in Washington, D.C. to photograph campaign doc

In [77]:
llm.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: llama2-7b-djl-deepspeed-ec66e
INFO:sagemaker:Deleting endpoint with name: llama2-7b-djl-deepspeed-ec66e
